# Kernel Operations

Load PENVM setup for deployed release 0.1.0.

In [ ]:
%run "penvm-0.1.0-setup.ipynb"

Set up World and boot "default" network.

In [ ]:
world = World(filename="world-1-local.penvm")
network = world.get_network()
network.boot()
machines = network.get_machines()
s = machines[0].get_session()

List available kernels.

In [ ]:
s.kernel.machine_list_kernels()
resp = s.get_response()
resp.payload.get("names")

Show kernel used by session (client-side).

In [ ]:
s.kernel

List client-side kernel interface methods.

In [ ]:
[name for name in dir(s.kernel) if not name.startswith("_")]

Get kernel op lists (server side).

In [ ]:
s.kernel.kernel_list_ops(s.kernel.name)
resp = s.get_response()
resp.payload.get("names")

Get kernel state.

In [ ]:
s.kernel.kernel_get_state(s.kernel.name)
resp = s.get_response()
resp.payload.get("state")

Define a new kernel op.

In [ ]:
code = '''
from penvm.kernels.base.server import Op
from penvm.lib.message import ErrorResponse, OkResponse, Payload


class Hello(Op):
    """Send "hello" in the `message` field in the response.

    Attributes: Request:
        -: Unused.

    Attributes: OkResponse:
        message (str): "hello".
    """

    def run(self, ctxt, req):
        try:
            resp = OkResponse(
                payload=Payload({"message": "hello"}),
                refmsg=req,
            )
            return resp
        except Exception as e:
            self.logger.error(f"{e}")
            return ErrorResponse(
                "exception {e}",
                refmsg=req,
            )
''' 

Add the new kernel op to the "default" kernel.

In [ ]:
_ = s.kernel.kernel_register_op("default", "hello", "Hello", code)

Check that the new kernel op is registered.

In [ ]:
s.kernel.kernel_list_ops("default")
resp = s.get_response()
resp.payload.get("names")

Call new kernel op.

Note: Since only the server-side was added, the request needs to be crafted by hand. In this case it is simple since there are no arguments required.

In [ ]:
s.newput_request("hello")
resp = s.get_response()
resp

In [ ]:
resp.payload.get("message")

Unregister the new kernel op.

In [ ]:
_ = s.kernel.kernel_unregister_op("default", "hello")

Check that the op is no longer registered.

In [ ]:
s.kernel.kernel_list_ops("default")
resp = s.get_response()
resp.payload.get("names")

Clean up.

In [ ]:
world.shutdown()